In [1]:
import os

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import cv2
import tensorflow as tf

In [4]:
def setStateNormal(pred):
    ret = None
    if pred>0.5:
        ret = "Crime"
    else:
        ret = "Normal"
        
    return f"{ret} - {round(pred*100, 4)}%"


def ShowPredictions(buffer_size:int=90):

    
    buffer = []
    prev_pred = "State: NaN"
    
    color = {
        1: (255, 0, 0), # blue 
        2: (0, 255, 0), # green
        3: (0, 0, 255), # red
        4: (106, 0, 0), # dark blue
        5: (0, 97, 0),  # dark green
        6: (0, 0, 96)   # dark red
    }
    
    multiplier=1
    outSize=(320*multiplier, 240*multiplier)

    cur_color = color[2]

    cap = cv2.VideoCapture(0)

    if cap.isOpened()==False:
        print("Either file not found or wrong codec used")

    while cap.isOpened():

        ret, frame = cap.read()

        if ret==True:

            # Make a copy of your frame and resize it to pass it to model
            out = frame.copy()

            out = cv2.cvtColor(out, cv2.COLOR_BGR2RGB)
            out = cv2.resize(out, (320, 240)).astype("float32")
            pred = model.predict(np.expand_dims(out, axis=0))[0][0]

            # if the buffer is not full yet, show the current frames prediction alone
            if len(buffer)<buffer_size:
                buffer.append(pred)
                state = setStateNormal(pred)
            # else show the rolling avg of the previous buffer_size frame's prediction 
            else:
                buffer.pop(0)
                buffer.append(pred)
                state = setStateNormal(sum(buffer)/buffer_size)
            prev_pred = state

            frame = cv2.putText(frame, state, (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, cur_color, 2)

            frame = cv2.resize(frame, outSize)
            cv2.imshow('frame',frame)

            if cv2.waitKey(1) & 0xFF==ord('q'):
                print(f"Prediction at last frame:\n")
                plt.imshow(frame)
                break

        else:
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
model = tf.keras.models.load_model(r"../Models/model_x-y-21.h5", compile=False)

In [6]:
# ShowPredictions()

In [9]:
import tkinter as tk
from tkinter import filedialog
import cv2
from PIL import ImageTk, Image
from modell import *
import re

window = tk.Toplevel()
window.geometry("1500x1000")
window.title("Crime Anomaly Detection using Deep Learning")

video_file = "vid2.mp4"
cap = cv2.VideoCapture(video_file)

canvas = tk.Canvas(window, width=1500, height=1000)
canvas.pack()

def update_background():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (1500, 1000))  
        image = ImageTk.PhotoImage(image=Image.fromarray(frame))
        canvas.create_image(0, 0, anchor="nw", image=image)
        canvas.image = image
        window.after(1, update_background)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  
        window.after(1, update_background)

update_background()

title_label = tk.Label(window, text="Crime Anomaly Detection using Deep Learning", font=("Arial", 44, "bold"))
title_label.place(relx=0.5, rely=0.1, anchor="center")

button = tk.Button(window, text="Real Time", command=ShowPredictions, font=("Arial", 30, "bold"), width=20, height=1)
button.place(relx=0.7, rely=0.95, anchor="s")

button1 = tk.Button(window, text="Upload Video", command=ShowPredictionss, font=("Arial", 30, "bold"), width=20, height=1)
button1.place(relx=0.3, rely=0.95, anchor="s")


window.mainloop()

Either file not found or wrong codec used
